### Do we have enough physiological data?
1. Reading all of Arash's data
2. Multivariate KDE on Arash's Data
3. Some functions for calculating Divergence
4. Running divergence on Arahs's data
5. Performing the same for Participant #13 different days data
6. Optional visualization for distributions
7. KDE visualization for paper
8. Same process for participant #1

In [ ]:
#1. Reading all of Arash's data

import pandas as pd 
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

hr = pd.read_csv("C:/Users/Arsalan/Desktop/both hands and car/raw/Smartwatch_HeartRateDatum.csv")
dev_dic = {"f60691a313420a4e":"left" , "39ca51c16b9ec429":"right"}
devId_vals = hr["DeviceId"].unique()
#

hr["Timestamp"] = pd.to_datetime(hr["Timestamp"])
#hr = hr[hr["Timestamp"]<"2020-08-22 02:15:20+00:00"]
hr = hr.set_index("Timestamp")
hr = hr.groupby(["DeviceId"]).resample('1000L').mean()
hr = hr.reset_index()
hr["HR"] = hr["HR"].bfill().ffill()
for val in devId_vals:
    hr_temp = hr[hr["DeviceId"] == val]
    x=[]
    group_labels =[]
    for i in range(20):
        x.append(hr_temp["HR"][0:(i+1)*900])
        group_labels.append(i)

    fig = ff.create_distplot(x,group_labels,show_hist=False)
    fig.show()

In [ ]:
#2. Multivariate KDE on Arash's Data

import pandas as pd 
import numpy as np
from tqdm import tqdm

df_ultimate = []

sensors = ["Smartwatch_AccelerometerDatum","Smartwatch_GyroscopeDatum","Smartwatch_HeartRateDatum",
           "Smartwatch_Light"]#,"Smartwatch_PPGDatum"]

hr = pd.read_csv("C:/Users/Arsalan/Desktop/both hands and car/raw/Smartwatch_HeartRateDatum.csv")
dev_dic = {"f60691a313420a4e":"left" , "39ca51c16b9ec429":"right"}
devId_vals = hr["DeviceId"].unique()

dfs = [pd.read_csv(("C:/Users/Arsalan/Desktop/both hands and car/raw/"+sensor+'.csv'), parse_dates=['Timestamp']) for sensor in sensors]
for id in tqdm(devId_vals):
    if id=="f60691a313420a4e":
        current_df = [df[df['DeviceId']==id] for df in dfs]
        current_df_merged = current_df[0]
        for df in current_df[1:]:
            current_df_merged = current_df_merged.merge(df,on=['Timestamp', 'DeviceId'], how='outer')
        current_df_merged = current_df_merged.sort_values(by = "Timestamp")
        current_df_merged["diff"] = ((current_df_merged["Timestamp"] - current_df_merged["Timestamp"].shift()).fillna(pd.Timedelta(seconds=0)))/np.timedelta64(1, 's')
        current_df_merged["group"] = current_df_merged["diff"].ge(5).cumsum()
        current_df_merged = current_df_merged.set_index("Timestamp")
        current_df_merged = current_df_merged.groupby(['DeviceId',"group"]).resample('2000L').mean()
        #print(current_df_merged)
        df_ultimate.append(current_df_merged)
df_ultimate=pd.concat(df_ultimate, axis=0)
df_ultimate.drop(columns=["group"],inplace=True)
df_ultimate = df_ultimate.reset_index()
df_ultimate = df_ultimate.bfill()
df_ultimate.dropna(how="any",inplace=True)

In [ ]:
#3. Some functions for calculating divergence
#functions for calculating the divergence

def compute_probs(data, n=10): 
    h, e = np.histogram(data, n)
    p = h/data.shape[0]
    return e, p

def support_intersection(p, q): 
    sup_int = list(
                filter(
                    lambda x: (x[0]!=0) & (x[1]!=0), zip(p, q)
                )
    )
    return sup_int

def get_probs(list_of_tuples): 
    p = np.array([p[0] for p in list_of_tuples])
    q = np.array([p[1] for p in list_of_tuples])
    return p, q

def kl_divergence(p, q): 
    return np.sum(p*np.log(p/q))

def js_divergence(p, q):
    m = (1./2.)*(p + q)
    return (1./2.)*kl_divergence(p, m) + (1./2.)*kl_divergence(q, m)

def compute_kl_divergence(train_sample, test_sample, n_bins=10): 
    """
    Computes the KL Divergence using the support 
    intersection between two different samples
    """
    e, p = compute_probs(train_sample, n=n_bins)
    _, q = compute_probs(test_sample, n=e)

    list_of_tuples = support_intersection(p, q)
    p, q = get_probs(list_of_tuples)
    
    return kl_divergence(p, q)

def compute_js_divergence(train_sample, test_sample, n_bins=10): 
    """
    Computes the JS Divergence using the support 
    intersection between two different samples
    """
    e, p = compute_probs(train_sample, n=n_bins)
    _, q = compute_probs(test_sample, n=e)
    
    list_of_tuples = support_intersection(p,q)
    p, q = get_probs(list_of_tuples)
    
    return js_divergence(p, q)

In [ ]:
#4. Running divergence on Arahs's data

from sklearn.neighbors import KernelDensity
import numpy as np
from scipy.stats import entropy

df_cleaned = df_ultimate[df_ultimate["group"] == 2]
X = df_cleaned.drop(columns=["DeviceId","group","Timestamp","diff"]).values
slice = 500
for i in range(5):
    temp_x_1 = X[0:(i+1) *(slice),:]
    temp_x_2 = X[0:(i+2) *slice,:]
    kde_1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(temp_x_1)
    kde_2 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(temp_x_2)
    est_1 = kde_1.score_samples(temp_x_1)
    est_2 = kde_1.score_samples(temp_x_2)
    ent = compute_kl_divergence(est_1,est_2,n_bins=10)
    print(ent)

In [ ]:
#5. Performing the same for Participant #13 different days data

import pandas as pd 
import pandas as pd 
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.neighbors import KernelDensity
import numpy as np
from scipy.stats import entropy
from tqdm import tqdm
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

hr = pd.read_csv("H:/13/final_HR.csv")

hr["Timestamp"] = pd.to_datetime(hr["Timestamp"])

hr = hr.set_index("Timestamp")
hr = hr.groupby(["video_name"]).resample('1000L').mean()
hr = hr.reset_index()
hr["HR"] = hr["HR"].bfill().ffill()

list_of_divs = []
X = (hr["HR"].values).reshape(-1,1)
slice = 250
for i in tqdm(range(147)):
    temp_x_1 = X[0:(i+1) *(slice)]
    temp_x_2 = X[0:(i+2) *slice]
    kde_1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(temp_x_1)
    kde_2 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(temp_x_2)
    est_1 = kde_1.score_samples(temp_x_1)
    est_2 = kde_1.score_samples(temp_x_2)
    ent = compute_kl_divergence(est_1,est_2,n_bins=10)
    list_of_divs.append(ent)
    #print(ent)
dic_of_divs = {}
for i in range(147):
    dic_of_divs[i*250] = list_of_divs[i]
print(dic_of_divs)
df = pd.DataFrame(list(dic_of_divs.items()),columns = ['dp','KDE'])
df["time"] = df["dp"]/3600    

#df = pd.read_csv("D:/Google Drive/UVA_BRAIn_Lab/Arash/Papers/IEEE ACCESS - Dataset Paper/kde.csv")

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=df["time"], y=df["KDE"],name= "Three Month of HR",line=dict(width=4,color='black')))
fig1.update_layout(    
    {    
        'yaxis':{'title':"<b> KL divergence",'showgrid':False,'linecolor':'black'},
        'xaxis':{'title':"<b>Time (hr)",'showgrid':False,'linecolor':'black'},
        'height':600,
        'width':600,
        'font':dict(size=14),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
    }
)
fig1.show()

In [ ]:
#6. Optional visualization for distributions
import math
x=[]
group_labels =[]
for i in range(10):
    x.append(hr["HR"][0:(i+1)*3600])
    group_labels.append(math.floor(i*12000/3600)
                       )
fig = ff.create_distplot(x,group_labels,show_hist=False)
fig.show()

In [ ]:
#7. KDE visualization for paper

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio

channels = ['HR']

files = ["final_hr.csv"]
df = pd.read_csv(files[0])
df['day'] = df['day'].apply(str)
df = df[df["type"]=="work to home"]
df = df.sort_values(by=['day', 'Timestamp'])
#df = df.set_index('Timestamp')

def time_corrector(x):
    y = x[4:6]+"/"+x[6:]+"/"+x[0:4]
    return y
df['correct_day'] = df['day'].apply(time_corrector)
df['day'] = df['correct_day']
days = df.day.unique()
#outside_events = df.event_outside.unique()



#inside_events = inside_events[:-1]



for channel in channels:
#     plt.figure(figsize=(20,20))
#     plt.title(channel)
    
    datas = []
    labels = []

    for day in days:
        df2 = df[df['day']==day].copy()
        #df2['day'] = df2['day'].apply('str')
        #sns.kdeplot(df2[channel])
        #plot_df(df2, event)
        if df2[channel].values.shape[0]>100:
            datas.append(np.random.choice(df2[channel].values, size=df2[channel].values.shape[0]))
            labels.append(str(day))
        
    
    fig1 = ff.create_distplot(datas, labels,show_curve=True, show_rug=False,show_hist=False)
    #fig1.update_layout(title=channel) 
    fig1.update_layout(  
    {    
        'xaxis':{'title':"<b>HR (bpm)",'linecolor':'black'},
        'yaxis':{'title':"<b>Probability",'linecolor':'black'},
        'height':1000,
        'width':1500,
        'font':dict(size=34),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
    
        
    }
    )
    fig1.update_layout(legend=dict(
    yanchor="top",
    xanchor="right",
        ))
    fig1.update_traces(
    line=dict( width=10),
    selector=dict(type="scatter", mode="lines"))


    fig1.show()

    #fig2.show()
    
    # Within people variability
pio.write_image(fig1, 'distributions_hr.pdf', width=1500, height=1000)

In [ ]:
#8. Same process for participant #1

import pandas as pd 
import numpy as np
from tqdm import tqdm

df_ultimate = []

sensors = ["Smartwatch_AccelerometerDatum","Smartwatch_GyroscopeDatum","Smartwatch_HeartRateDatum",
           "Smartwatch_Light"]#,"Smartwatch_PPGDatum"]

hr = pd.read_csv("C:/Users/Arsalan/Desktop/both hands and car/Arsalan_data/Smartwatch_HeartRateDatum.csv")
devId_vals = hr["DeviceId"].unique()

dfs = [pd.read_csv(("C:/Users/Arsalan/Desktop/both hands and car/Arsalan_data/"+sensor+'.csv'), parse_dates=['Timestamp']) for sensor in sensors]
for id in tqdm(devId_vals):
    current_df = [df[df['DeviceId']==id] for df in dfs]
    current_df_merged = current_df[0]
    for df in current_df[1:]:
        current_df_merged = current_df_merged.merge(df,on=['Timestamp', 'DeviceId'], how='outer')
    current_df_merged = current_df_merged.sort_values(by = "Timestamp")
    current_df_merged["diff"] = ((current_df_merged["Timestamp"] - current_df_merged["Timestamp"].shift()).fillna(pd.Timedelta(seconds=0)))/np.timedelta64(1, 's')
    current_df_merged["group"] = current_df_merged["diff"].ge(5).cumsum()
    current_df_merged = current_df_merged.set_index("Timestamp")
    current_df_merged = current_df_merged.groupby(['DeviceId',"group"]).resample('2000L').mean()
    #print(current_df_merged)
    df_ultimate.append(current_df_merged)
        
df_ultimate=pd.concat(df_ultimate, axis=0)
df_ultimate.drop(columns=["group"],inplace=True)
df_ultimate = df_ultimate.reset_index()
df_ultimate = df_ultimate.bfill()
#df_ultimate.dropna(how="any",inplace=True)

from sklearn.neighbors import KernelDensity
import numpy as np
from scipy.stats import entropy

results={}
df_cleaned = df_ultimate.copy()
X = df_cleaned.drop(columns=["DeviceId","group","Timestamp","diff"]).values
slice = 10000
for i in tqdm(range(500)):
    temp_x_1 = X[0:(i+1) *(slice),:]
    temp_x_2 = X[0:(i+2) *slice,:]
    kde_1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(temp_x_1)
    kde_2 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(temp_x_2)
    est_1 = kde_1.score_samples(temp_x_1)
    est_2 = kde_1.score_samples(temp_x_2)
    ent = compute_kl_divergence(est_1,est_2,n_bins=10)
    results[i] = ent